In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
df = pd.read_excel('/content/Datasets _ NLP based tagging solution078466f.xlsx',sheet_name='Training Data ')
df.head()

,Exception (input),Queue,Process,Exception Category (ouput)
0,Invite not found in both CBS mailbox,Queue-12,Process-9,Business Exception
1,Interviewer xxxx mail ID not found in invite.,Queue-12,Process-9,Business Exception
2,Invite not found in both TAX and PAS mailbox,Queue-12,Process-9,Business Exception
3,InternalFailed to evaluate expression 'Replace...,Queue-14,Process-11,System Exception
4,Could not execute code stage because exception...,Queue-16,Process-13,System Exception


In [ ]:
#preprocessing steps

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing punctuation
    - Lowering text
    """
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)  
    # text = re.sub(r'[0-9]','', text)  
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

In [ ]:
#vectorization

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english", preprocessor=clean_text)
train = df['Exception (input)']

training_features = vectorizer.fit_transform(train)

In [ ]:
vectorizer.vocabulary_.items()

dict_items([('invite', 79), ('cbs', 26), ('mailbox', 98), ('interviewer', 78), ('xxxx', 183), ('mail', 96), ('id', 73), ('tax', 167), ('pas', 119), ('internalfailed', 77), ('evaluate', 51), ('expression', 55), ('replace', 138), ('mailbody', 97), ('item', 86), ('data', 40), ('collection', 33), ('execute', 54), ('code', 31), ('stage', 159), ('exception', 52), ('thrown', 172), ('given', 68), ('key', 91), ('present', 124), ('dictionary', 43), ('email', 46), ('failed', 57), ('read', 134), ('automatically', 19), ('set', 156), ('cleanup', 27), ('session', 155), ('end', 47), ('recruiter', 135), ('assurance', 18), ('occured', 112), ('accessing', 12), ('article', 17), ('page', 117), ('unable', 175), ('processed', 126), ('launch', 93), ('service', 154), ('site', 158), ('retries', 142), ('missing', 104), ('mandatory', 100), ('fields', 61), ('consulting', 34), ('internal', 76), ('new', 109), ('work', 181), ('queue', 130), ('items', 87), ('filtered', 64), ('output', 115), ('p2', 116), ('current', 39

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

model = LinearSVC()
model2 = MultinomialNB()
model3 = LogisticRegression()
model.fit(training_features[:40], df["Exception Category (ouput)"][:40])
model2.fit(training_features[:40], df["Exception Category (ouput)"][:40])
model3.fit(training_features[10:], df["Exception Category (ouput)"][10:])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = model.predict(training_features[40:])
y_pred2 = model2.predict(training_features[40:])
y_pred3 = model3.predict(training_features[0:10])

In [ ]:
print(y_pred3)

['Business Exception' 'Business Exception' 'Business Exception'
 'System Exception' 'System Exception' 'Business Exception'
 'Business Exception' 'Business Exception' 'Business Exception'
 'Business Exception']


In [ ]:
print(y_pred)

['System Exception' 'System Exception' 'System Exception'
 'System Exception' 'System Exception' 'System Exception'
 'Business Exception']


In [ ]:
print(y_pred2)

['System Exception' 'System Exception' 'System Exception'
 'System Exception' 'System Exception' 'System Exception'
 'Business Exception']


In [ ]:
print('Accuracy for Exception category using SVC: ',accuracy_score(df["Exception Category (ouput)"][40:],y_pred))
print('Accuracy for Exception category using Naive Bayes: ',accuracy_score(df["Exception Category (ouput)"][40:],y_pred2))
print('Accuracy for Exception category using Logistic Regression: ',accuracy_score(df["Exception Category (ouput)"][0:10],y_pred3))

Accuracy for Exception category using SVC:  0.8571428571428571
Accuracy for Exception category using Naive Bayes:  0.8571428571428571
Accuracy for Exception category using LOgistic Regression:  0.8


We have used Logistic Regression, Multinomial Naive-Bayes, and Support Vector Machine as our training models.

In [ ]:
df_out = pd.DataFrame(data = {'Input': train[40:], 'Output':y_pred})
df_out

,Input,Output
40,Failed to fetch data from RACE tool due to sit...,System Exception
41,Could not execute code stage because exception...,System Exception
42,Internal_Could not execute code stage because ...,System Exception
43,Failed to perform step 1 in Read Stage 'Read p...,System Exception
44,BBC Update failed for BBC ID- 330- The remote ...,System Exception
45,Failed to perform step 1 in Navigate Stage 'Cl...,System Exception
46,Error in populating result in output template ...,Business Exception


In [ ]:
df_out.to_excel('output.xlsx',index = False)